<br>
    <center>МИНИСТЕРСТВО НАУКИ И ВЫСШЕГО ОБРАЗОВАНИЯ</center>
    <center>федеральное государственное автономное образовательное учреждение высшего образования </center> <center>«Самарский национальный исследовательский университет имени академика С.П. Королева»</center>
    <center>(Самарский университет)</center> </br>

<br/>

<br>
<center>Институт 	     информатики и кибернетики</center>                                                   	  
<center>Кафедра 	     технической кибернетики</center>                                                              	
</br>
<br/>

<br>

<br/>
<br/>
<br/>
<br>
<center>ОТЧЕТ</center>
<center>по лабораторной работе №2</center>

<center>«Обработка данных с использование Kafka»</center>
<br/>
<center>по дисциплине <strong>«Большие данные»</strong></center>
<br/>
<center></center>
</br>
<br/>
<br/>
<br/>



<br>
<p style="text-align:right;">Выполнил: Фамилия И.О.
<br>613х-010402D
<br>    
<br>Преподаватель: Попов С.Б.
</p>
<br/>
<br/>
<br/>
    <br/>
<br/>
<br/>
<center>Самара 2025</center>


### Экспорт необходимых модулей

In [ ]:
from kafka import KafkaProducer
from kafka import KafkaConsumer
from kafka.errors import KafkaError
from kafka.structs import TopicPartition

from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions


### Определение списка серверов kafka

In [ ]:
bootstrap_servers = 'localhost:9092'

## Реализация схемы с одним обработчиком (one consumer)

### Создание топика для схемы *one consumer*

In [ ]:
admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

topic1 = 'rides1-events'

topic_list = []
topic_list.append(NewTopic(name=topic1, num_partitions=1, replication_factor=1))
admin_client.create_topics(new_topics=topic_list, validate_only=False)

print(admin_client.list_topics())

admin_client.close()

### Реализация источника данных (producer)

1. Создаём экземпляр класса KafkaProducer (не забываем определить сериализаторы для key- и value-составляющих в сообщении)
2. Подключаем доступ на чтение из файла *riders.csv*
3. Читаем и пропускаем первую строку-заголовок
4. В цикле считываем строки из csv-файла и отправлем их в топик
5. Завершаем работу объекта KafkaProducer

In [ ]:
producer = KafkaProducer(bootstrap_servers=[bootstrap_servers],
                        key_serializer= lambda key: key.encode('utf-8'),
                        value_serializer= lambda value: value.encode('utf-8'))


In [ ]:
producer.close()

### Реализация обработчика данных (consumer)

1. Создаём экземпляр класса KafkaConsumer (не забываем в параметрах определить параметр *consumer_timeout_ms* и десериализаторы для key- и value-составляющих в сообщении)
2. Создаём массивы для накопления значений trip_count, passenger_count, trip_distance, total_amount с нулевыми значениями и соответствующим типом данных
3. В цикле получения сообщений объекта KafkaConsumer:
   * разделяем получаемую value-строку на отдельные элементы,
   * выделяем значение VendorID или используем key, если источник передаёт его в сообщении,
   * обновляем значения в массивах-накопителях, используя (VendorID - 1) как индекс соответствующего массива 
4. Распечатываем отдельно для каждого перевозчика:
   * количество поездок(trip_count),
   * число перевезённых пассажиров(passenger_count),
   * расстояние поездок(trip_distance)
   * сумму оплаты(total_amount)
5. Завершаем работу объекта KafkaProducer

In [ ]:
consumer = KafkaConsumer(topic1,
                         auto_offset_reset='earliest',
                         consumer_timeout_ms=2000,
                         key_deserializer= lambda key: int(key.decode('utf-8')),
                         value_deserializer= lambda value: value.decode('utf-8'),
                         bootstrap_servers=[bootstrap_servers])


In [ ]:
consumer.close()

## Реализация схемы с двумя обработчиками (two consumers launched in a group)

### Создание топика для схемы *two consumer*


!!! Не забываем указать количество разделов с помощью параметра *num_partitions=2*

### Реализация источника данных (producer)

1. Создаём экземпляр класса KafkaProducer (не забываем определить сериализаторы для key- и value-составляющих в сообщении)
2. Подключаем доступ на чтение из файла *riders.csv*
3. Читаем и пропускаем первую строку-заголовок
4. В цикле считываем строки из csv-файла:
    * преобразуем строку в список элементов (используем метод split(','))
    * выделяем значение VendorID
    * отправлем сообщение в топик с key=*VendorID* и value=*cтрока файла*
6. Завершаем работу объекта KafkaProducer

## Полезные утилиты

### Просмотр списка топиков

In [ ]:
from kafka.errors import KafkaError
from kafka.structs import TopicPartition
from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions

bootstrap_servers = 'localhost:9092'

admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

print(admin_client.list_topics())

admin_client.close()

### Удаление топика

In [ ]:
from kafka.errors import KafkaError
from kafka.structs import TopicPartition
from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions

bootstrap_servers = 'localhost:9092'

# ниже введите имя топика из списка, распечатанного предыдущей ячейкой
topic_for_del = '__consumer_offsets'

admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)
admin_client.delete_topics([topic_for_del])

print(admin_client.list_topics())

admin_client.close()

### Просмотр списка групп

In [ ]:
from kafka.errors import KafkaError
from kafka.structs import TopicPartition
from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions

bootstrap_servers = 'localhost:9092'

admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

print(admin_client.list_consumer_groups())

admin_client.close()

### Просмотр описания заданной группы (в данном случае – *rides2*)

In [ ]:
from kafka.errors import KafkaError
from kafka.structs import TopicPartition
from kafka.admin import KafkaAdminClient, NewTopic, NewPartitions

bootstrap_servers = 'localhost:9092'

admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)

print(admin_client.describe_consumer_groups(group_ids=['rides2']))

admin_client.close()